## Neural networks assignment- Q2

#### Q) Predicting turbine energy yield (TEY) using ambient variables as features

In [1]:
# to supress the warnings
import warnings
warnings.filterwarnings('ignore')

In [39]:
#Import the required libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
# Read the data
data = pd.read_csv('gas_turbines.csv')
data.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


In [4]:
#more information about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15039 entries, 0 to 15038
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      15039 non-null  float64
 1   AP      15039 non-null  float64
 2   AH      15039 non-null  float64
 3   AFDP    15039 non-null  float64
 4   GTEP    15039 non-null  float64
 5   TIT     15039 non-null  float64
 6   TAT     15039 non-null  float64
 7   TEY     15039 non-null  float64
 8   CDP     15039 non-null  float64
 9   CO      15039 non-null  float64
 10  NOX     15039 non-null  float64
dtypes: float64(11)
memory usage: 1.3 MB


In [5]:
# Describing the data
data.describe()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
count,15039.000000,15039.00000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000
mean,17.764381,1013.19924,79.124174,4.200294,25.419061,1083.798770,545.396183,134.188464,12.102353,1.972499,68.190934
std,7.574323,6.41076,13.793439,0.760197,4.173916,16.527806,7.866803,15.829717,1.103196,2.222206,10.470586
min,0.522300,985.85000,30.344000,2.087400,17.878000,1000.800000,512.450000,100.170000,9.904400,0.000388,27.765000
25%,11.408000,1008.90000,69.750000,3.723900,23.294000,1079.600000,542.170000,127.985000,11.622000,0.858055,61.303500
50%,18.186000,1012.80000,82.266000,4.186200,25.082000,1088.700000,549.890000,133.780000,12.025000,1.390200,66.601000
75%,23.862500,1016.90000,90.043500,4.550900,27.184000,1096.000000,550.060000,140.895000,12.578000,2.160400,73.935500
max,34.929000,1034.20000,100.200000,7.610600,37.402000,1100.800000,550.610000,174.610000,15.081000,44.103000,119.890000


In [6]:
# Splitting the data
X = data.loc[:,['AT','AP','AH','AFDP','GTEP','TIT','TAT','CDP','CO','NOX']]
y = data.loc[:,['TEY']]

#### Standardizing the data

In [7]:
sc = StandardScaler()
X_array  = sc.fit_transform(X)
y_array = sc.fit_transform(y)

In [8]:
pd.DataFrame(X_array).describe()

,0,1,2,3,4,5,6,7,8,9
count,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04
mean,-2.320107e-16,-1.925280e-14,1.844983e-16,3.810001e-16,1.107344e-16,-2.324212e-15,1.744899e-15,3.640356e-16,1.953355e-17,-6.862579e-17
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00
min,-2.276462e+00,-4.266288e+00,-3.536594e+00,-2.779497e+00,-1.806771e+00,-5.021933e+00,-4.188141e+00,-1.992416e+00,-8.874862e-01,-3.861033e+00
25%,-8.392292e-01,-6.706510e-01,-6.796337e-01,-6.266930e-01,-5.091458e-01,-2.540512e-01,-4.101146e-01,-4.354335e-01,-5.015202e-01,-6.578107e-01
50%,5.566605e-02,-6.227861e-02,2.277844e-01,-1.854065e-02,-8.075681e-02,2.965544e-01,5.712570e-01,-7.011925e-02,-2.620452e-01,-1.518527e-01
75%,8.051309e-01,5.772924e-01,7.916582e-01,4.612196e-01,4.228638e-01,7.382490e-01,5.928675e-01,4.311680e-01,8.455882e-02,5.486567e-01
max,2.266234e+00,3.275970e+00,1.528011e+00,4.486233e+00,2.871006e+00,1.028678e+00,6.627839e-01,2.700105e+00,1.895949e+01,4.937717e+00


#### Tuning of Hyperparameters :- Batch Size and Epochs

In [9]:
def basemodel():
    model = Sequential()
    model.add(Dense(10, input_dim = 10,activation ='relu'))
    model.add(Dense(4,activation= 'relu'))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss='mse', optimizer = 'adam')
    return model

In [10]:
model = KerasRegressor(build_fn=basemodel,verbose =0)
# Define the grid search parameters
batch_size = [5,10,15]
epochs = [10,50,100]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model ,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_array,y_array)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=5, epochs=10.....................................
[CV 1/5; 1/9] END .....batch_size=5, epochs=10;, score=-0.686 total time=  40.1s
[CV 2/5; 1/9] START batch_size=5, epochs=10.....................................
[CV 2/5; 1/9] END .....batch_size=5, epochs=10;, score=-0.437 total time=  37.9s
[CV 3/5; 1/9] START batch_size=5, epochs=10.....................................
[CV 3/5; 1/9] END .....batch_size=5, epochs=10;, score=-0.502 total time=  37.6s
[CV 4/5; 1/9] START batch_size=5, epochs=10.....................................
[CV 4/5; 1/9] END .....batch_size=5, epochs=10;, score=-0.485 total time=  37.6s
[CV 5/5; 1/9] START batch_size=5, epochs=10.....................................
[CV 5/5; 1/9] END .....batch_size=5, epochs=10;, score=-0.682 total time=  38.1s
[CV 1/5; 2/9] START batch_size=5, epochs=50.....................................
[CV 1/5; 2/9] END .....batch_size=5, epochs=50;, 

In [11]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.5581401646137237, using {'batch_size': 10, 'epochs': 100}
-0.5584311306476593,0.10494539179610432 with: {'batch_size': 5, 'epochs': 10}
-0.5583658576011657,0.10494892456473533 with: {'batch_size': 5, 'epochs': 50}
-0.5582134127616882,0.10480869008786008 with: {'batch_size': 5, 'epochs': 100}
-0.5586721658706665,0.10489552100211819 with: {'batch_size': 10, 'epochs': 10}
-0.5582415401935578,0.10489922688715839 with: {'batch_size': 10, 'epochs': 50}
-0.5581401646137237,0.10490166552440919 with: {'batch_size': 10, 'epochs': 100}
-0.5588429152965546,0.10494782863907733 with: {'batch_size': 15, 'epochs': 10}
-0.5582571923732758,0.1049337309567765 with: {'batch_size': 15, 'epochs': 50}
-0.5581496119499206,0.10498314244003577 with: {'batch_size': 15, 'epochs': 100}


In [12]:
grid_result.best_score_

-0.5581401646137237

In [13]:
grid_result.best_params_

{'batch_size': 10, 'epochs': 100}

In [ ]:
#### Tuning of Hyperparameters :- learning rate and dro[pout rate]

In [14]:
from keras.layers import Dropout

# Defining the model

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(10,input_dim = 10,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4,input_dim = 10,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss = 'mean_squared_error',optimizer = 'Adam')
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 5,epochs = 50)

# Define the grid search parameters

learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]

# Make a dictionary of the grid search parameters

param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_array,y_array)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 1/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.686 total time= 3.2min
[CV 2/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 2/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.437 total time= 3.2min
[CV 3/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 3/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.501 total time= 3.3min
[CV 4/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 4/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.485 total time= 3.2min
[CV 5/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 5/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.682 total time= 3.3min
[CV 1/5; 2/9] START dropout_rate=0.0, learning_rate=0.01........................
[CV 

In [15]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.5580870866775512, using {'dropout_rate': 0.0, 'learning_rate': 0.001}
-0.5580870866775512,0.1049326175494691 with: {'dropout_rate': 0.0, 'learning_rate': 0.001}
-0.5582700133323669,0.1048858955468957 with: {'dropout_rate': 0.0, 'learning_rate': 0.01}
-0.5581759750843048,0.10482607593926076 with: {'dropout_rate': 0.0, 'learning_rate': 0.1}
-0.5589388072490692,0.10546260300000063 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
-0.5587832152843475,0.10475629201845782 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
-0.5583952784538269,0.10482878747109459 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
-0.5594920337200164,0.10474106050356645 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
-0.5588758587837219,0.10460235508530728 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
-0.5599914371967316,0.10388366741526837 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}


#### Tuning of Hyperparameters:- Activation Function and Kernel Initializer

In [16]:
# Defining the model

def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(8,input_dim = 10,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 10,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss = 'mean_squared_error',optimizer = 'Adam')
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 5,epochs = 50)

# Define the grid search parameters
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_array,y_array)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 1/5; 1/12] END activation_function=softmax, init=uniform;, score=-0.687 total time= 4.3min
[CV 2/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 2/5; 1/12] END activation_function=softmax, init=uniform;, score=-0.441 total time= 4.1min
[CV 3/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 3/5; 1/12] END activation_function=softmax, init=uniform;, score=-0.504 total time= 4.2min
[CV 4/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 4/5; 1/12] END activation_function=softmax, init=uniform;, score=-0.490 total time= 4.2min
[CV 5/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 5/5; 1/12] END activation_function=softmax, init=uniform;, score=-0.685 total time= 4.2min
[CV 1/5; 2/12] START activation_function=softmax, init=norm

[CV 3/5; 10/12] END activation_function=linear, init=uniform;, score=-0.502 total time= 4.2min
[CV 4/5; 10/12] START activation_function=linear, init=uniform..................
[CV 4/5; 10/12] END activation_function=linear, init=uniform;, score=-0.486 total time= 4.1min
[CV 5/5; 10/12] START activation_function=linear, init=uniform..................
[CV 5/5; 10/12] END activation_function=linear, init=uniform;, score=-0.683 total time= 4.1min
[CV 1/5; 11/12] START activation_function=linear, init=normal...................
[CV 1/5; 11/12] END activation_function=linear, init=normal;, score=-0.687 total time= 4.8min
[CV 2/5; 11/12] START activation_function=linear, init=normal...................
[CV 2/5; 11/12] END activation_function=linear, init=normal;, score=-0.438 total time= 4.4min
[CV 3/5; 11/12] START activation_function=linear, init=normal...................
[CV 3/5; 11/12] END activation_function=linear, init=normal;, score=-0.502 total time= 4.3min
[CV 4/5; 11/12] START activa

In [17]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.559031754732132, using {'activation_function': 'linear', 'init': 'uniform'}
-0.5614878714084626,0.10374859034181723 with: {'activation_function': 'softmax', 'init': 'uniform'}
-0.5611711144447327,0.10585347782579181 with: {'activation_function': 'softmax', 'init': 'normal'}
-0.5610242903232574,0.10358185050505364 with: {'activation_function': 'softmax', 'init': 'zero'}
-0.559269517660141,0.1048255057308508 with: {'activation_function': 'relu', 'init': 'uniform'}
-0.5594012439250946,0.10406878034007588 with: {'activation_function': 'relu', 'init': 'normal'}
-1.0116025447845458,0.28296185438968774 with: {'activation_function': 'relu', 'init': 'zero'}
-0.5595818400382996,0.10456897578647592 with: {'activation_function': 'tanh', 'init': 'uniform'}
-0.5595149576663971,0.1046886231868828 with: {'activation_function': 'tanh', 'init': 'normal'}
-1.0115487933158875,0.28305318649342687 with: {'activation_function': 'tanh', 'init': 'zero'}
-0.559031754732132,0.10477300096303928 with: {'

#### Tuning of Hyperparameter :-Number of Neurons in activation layer

In [18]:
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 10,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss = 'mean_squared_error',optimizer = 'Adam')
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 5,epochs = 50)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_array,y_array) 

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 1/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.688 total time= 4.1min
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 2/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.440 total time= 4.1min
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 3/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.503 total time= 4.1min
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 4/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.489 total time= 4.2min
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 5/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.685 total time= 4.1min
[CV 1/5; 2/9] START neuron1=4, neuron2=4........................................
[CV 1/5; 2/9] END ........neuron1=4, neuron2=4;, 

In [19]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.559312641620636, using {'neuron1': 16, 'neuron2': 8}
-0.5609307885169983,0.10470615317435386 with: {'neuron1': 4, 'neuron2': 2}
-0.560339766740799,0.10492846192997078 with: {'neuron1': 4, 'neuron2': 4}
-0.5600373923778534,0.10463744948202593 with: {'neuron1': 4, 'neuron2': 8}
-0.5602546572685242,0.104670363206434 with: {'neuron1': 8, 'neuron2': 2}
-0.5597877383232117,0.10479683337831196 with: {'neuron1': 8, 'neuron2': 4}
-0.5593795239925384,0.10468535984573756 with: {'neuron1': 8, 'neuron2': 8}
-0.5601932466030121,0.10459467620424419 with: {'neuron1': 16, 'neuron2': 2}
-0.559420382976532,0.10470163576520046 with: {'neuron1': 16, 'neuron2': 4}
-0.559312641620636,0.10463602590658258 with: {'neuron1': 16, 'neuron2': 8}


#### Training model with optimum values of Hyperparameters

In [25]:
# Defining the model

def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 10,kernel_initializer = 'uniform',activation = 'linear'))
    model.add(Dropout(0.0))
    model.add(Dense(8,input_dim = 16,kernel_initializer = 'uniform',activation = 'linear'))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'sigmoid'))
    
    #adam = Adam(lr = 0.001) #sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss = 'mean_squared_error',optimizer = 'Adam')
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 10,batch_size = 5,epochs = 100)

# Fitting the model

model.fit(X_array,y_array)

# Predicting using trained model

y_predict = model.predict(X_array)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

ValueError: continuous is not supported

In [30]:
#Predicted values
y_predict

array([4.3105541e-07, 4.5774726e-07, 4.7827700e-07, ..., 8.9488559e-09,
       2.2263411e-08, 6.8635806e-08], dtype=float32)

In [34]:
#Converting the preicted standardized values
pred_val= sc.inverse_transform(y_predict)
pred_val

array([134.18848, 134.18848, 134.18848, ..., 134.18846, 134.18846,
       134.18846], dtype=float32)

In [35]:
#Converting the standardized y values to actaul y values
actual_val = sc.inverse_transform(y_array)
actual_val

array([[114.7 ],
       [114.72],
       [114.71],
       ...,
       [110.19],
       [110.74],
       [111.58]])

In [36]:
#Calculating the mean_squared_error
mean_squared_error(actual_val,pred_val)

140.00301794557598

In [38]:
#Calculating the Root_mean_squared_error
RMSE =sqrt(mean_squared_error(actual_val,pred_val))
RMSE

11.832287096989152